In [27]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# import
import pandas as pd
from pprint import pprint
from zen_garden.postprocess.results import Results
from pathlib import Path
from os import chdir

# set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# define import path
path_import = Path("../data/outputs/04_CCTS_WTE_Cement_01_optimal_design")

# define export path
path_export = Path("/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/04_CCTS_WTE_Cement_02_failure")

In [29]:
# get cost optimal solution
results = Results(path_import)

In [30]:
# get capacity addition and save as csv
results.get_df('capacity_addition', to_csv="capacity_addition")

technology     capacity_type  location               year
capture_amine  power          cement_1_CH            0        0.000000
                                                     1        0.000000
                                                     2       45.515506
                                                     3        0.000000
                                                     4        0.000000
                                                               ...    
truck_batch    power          waste_9_CH-waste_8_CH  5        0.000000
                                                     6        0.000000
                                                     7        0.000000
                                                     8        0.000000
                                                     9        0.000000
Name: capacity_addition, Length: 24570, dtype: float64

In [31]:
# get capacity addition csv
df = pd.read_csv('capacity_addition.csv')

In [32]:
# create existing capacity files

# find unique technologies
technology = df['technology'].unique()

for i in range(len(technology)):
     
    #Get table of one technology 
    tec = technology[i]
    x = df[df['technology'] == tec]
    
    #Set path
    parent_path = Path(path_export)
    path = [file for file in Path(parent_path).rglob(tec)]
    path = path[0]
    filepath = Path(str(path)+'/capacity_existing.csv')
    
    #Clean up table
    x = x.drop(columns='capacity_type')
    x = x.drop(columns='technology')
    x = x[x['capacity_addition'] != 0]
    x = x.rename(columns={'year': 'year_construction'})
    x = x.rename(columns={'capacity_addition': 'capacity_existing'})
    x['year_construction'] = x['year_construction'] + 2025 
    
    if  tec == 'biomethane_boiler':
        x['capacity_existing'] = x['capacity_existing'] / 1000
    
    #clean up table, differentiate between node and edge
    if path.parent.name == 'set_transport_technologies':
        x = x.rename(columns={'location': 'edge'})

    else:
        x = x.rename(columns={'location': 'node'})

    #Safe
    x.to_csv(filepath, index=False)